<a href="https://colab.research.google.com/github/maxim-chn/deep-learning/blob/maxim/src/preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of Contents


*   [Dependencies management](#dependecies-management)
*   [Project Config Init](#project-config-init)
*   [Connection Setup to Google Cloud Storage](#connection-setup-to-cloud-storage)
*   [Task Definition](#task-definition)
*   [Batch Definition](#batch-definition)
*   [Main Execution Path](#main-execution-path)

## Dependencies Management

In [1]:
# install dependencies

In [37]:
from io import BytesIO
import json
import multiprocessing
import os
from time import sleep
import requests

import matplotlib.pyplot as plt
import numpy as np

from google.cloud import storage as storage
from google.colab import auth as auth

from PIL import Image

[Back to Table of Contents](#table-of-contents)

## Project Config Init
We initiate configuration like location of buckets for unpreprocessed and processed images, machine limit in terms of parallel tasks, runtime parameters that define types of tasks etc.

In [14]:
FLOW = "classic" # Possibilities: "classic", "noise_framed"
VERSION = "maxim"
CONFIG_URL = f"https://raw.githubusercontent.com/maxim-chn/deep-learning/{VERSION}/config/project.json"

In [22]:
response = requests.get(CONFIG_URL)
if response.status_code == 200:
  project_config = response.json()
else:
  raise RuntimeError("Failed to download project config")
project_config["machine"]["tasks_number"] = multiprocessing.cpu_count()

In [23]:
BUCKETS = project_config["buckets"]
MACHINE = project_config["machine"]
MODEL_INPUT = project_config["model_input"]
PROJECT_ID = project_config["id"]

[Back to Table of Contents](#table-of-contents)

## Connection Setup to Google Cloud Storage
Our goal is to gain access to the *Google Cloud Filesystem*, which is [Google Buckets](https://console.cloud.google.com/storage/browser?hl=en&project=confident-trail-426114-e6) in the case of our project.

The interaction with the Google Cloud Filesystem is managed by the [google-cloud-storage module](https://cloud.google.com/python/docs/reference/storage/latest), initialized with the [google-cloud-storage's Client](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client)

In [6]:
auth.authenticate_user()
client = storage.Client(project=PROJECT_ID)

[Back to Table of Contents](#table-of-contents)

## Custom API Definition
This is utility methods and parameters defined specifically for the purpose of preprocessing.

### Image API

In [7]:
class ProcessedImage:
  num_of_zeros_in_name_prefix = 4

  def __init__(self, counter: int, tr_data: list = [], te_data: bytes = None):
    self._counter = counter
    self._counted_tr = 0
    self._init_name()
    self._init_tr_data(tr_data)
    self._init_te_data(te_data)

  @property
  def name(self) -> str:
    return self._name

  @property
  def te_data(self) -> dict:
    return self._te_data

  @property
  def tr_data(self) -> list:
    return self._tr_data

  def _init_name(self) -> None:
    if self._counter in [10, 100, 1000, 10000]:
      ProcessedImage.num_of_zeros_in_name_prefix -= 1
    self._name = "img_%s%s" % (''.join(['0' for _ in range(0, self.num_of_zeros_in_name_prefix)]), self._counter)

  def _init_te_data(self, data: bytes = None) -> None:
    self._te_data = None
    if data is None:
      return
    te_obj_name_prefix = f"{self.name}_te"
    self._te_data = {"name": f"{self.name}_te", "data": data}

  def _init_tr_data(self, data: list) -> None:
    self._tr_data = []
    if len(data) == 0:
      return
    for tr_obj_data in data:
      self._tr_data.append({"name": f"{self.name}_tr_{self._counted_tr}", "data": tr_obj_data})
      self._counted_tr += 1

class UnprocessedImage:
  images_counted = 0

  def __init__(self, name: str, data: bytes):
    self._data = data
    self._name = name
    self._update_counter()

  @property
  def counter(self) -> int:
    return self._counter

  @property
  def data(self) -> str:
    return self._data

  @property
  def name(self) -> str:
    dot_idx = self._name.find('.')
    if dot_idx != -1:
      return self._name[:dot_idx]
    return self._name

  def _update_counter(self):
    self._counter = UnprocessedImage.images_counted
    UnprocessedImage.images_counted += 1

In [8]:
def convert_bytes_to_image(data_bytes: bytes) -> Image:
  return Image.open(BytesIO(data_bytes))

def convert_image_to_bytes(image: Image) -> bytes:
  result = BytesIO()
  image.save(result, format="PNG")
  result.seek(0)
  return result.getvalue()

def convert_image_to_grayscale(image: Image) -> Image:
  return image.convert("L")

### BucketOfImages API

In [9]:
def get_batch_of_images(client: storage.Client, blob_idx_start: int, num_of_blobs: int) -> list:
  bucket = client.get_bucket(BUCKETS["unpreprocessed"]["name"])
  counted = 0
  result = []
  for index,blob in enumerate(bucket.list_blobs()):
    if index < blob_idx_start:
      continue
    if counted > num_of_blobs - 1:
      break
    counted += 1
    result.append(UnprocessedImage(blob.name, blob.download_as_bytes()))
  return result

def get_batches_of_images(client: storage.Client, num_of_batches: int, blob_idx_start: int, num_of_blobs: int) -> list:
  result = []
  while num_of_batches > 0:
    result.append(get_batch_of_images(client, blob_idx_start, num_of_blobs))
    blob_idx_start += num_of_blobs
    num_of_batches -= 1
  return result

def upload_blob(bucket: storage.Bucket, path: str, data: bytes) -> None:
  blob = bucket.blob(path)
  blob.upload_from_string(data, content_type="image/%s" % MODEL_INPUT["type"])

def persist_processed_image(bucket: storage.Bucket, processed_image: ProcessedImage) -> None:
  bucket_tr_name = BUCKETS["preprocessed"]["train"]["name"]
  for tr_obj in processed_image.tr_data:
    upload_blob(bucket, "%s/%s.png" % (bucket_tr_name, tr_obj["name"]), tr_obj["data"])
  if not processed_image.te_data is not None:
    return
  bucket_te_name = BUCKETS["preprocessed"]["test"]["name"]
  upload_blob(bucket, "%s/%s.png" % (bucket_te_name, processed_image.te_data["name"]), processed_image.te_data["data"])

def persist_batch_of_images(client: storage.Client, processed_batch: list) -> None:
  bucket = client.bucket(BUCKETS["preprocessed"]["name"])
  for processed_image in processed_batch:
    persist_processed_image(bucket, processed_image)

### Task API
A `Task` in the context of this Notebook is a piece of logic that performs the following operations on a batch of images:

*   Resize
*   Conversion to Grayscale
*   Crop
*   Rotation

The necessity for `Task` arises from [Global Interpreter Lock (GIL)](https://docs.python.org/3/glossary.html#term-global-interpreter-lock).  
Python doesn't support parallelism as well as other counterparts. Hence, we will create a `Task` for each batch of unpreprocessed images.

Module [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) will execute all of the tasks at the same time, thus bypassing GIL.

Our primary limitation is `bottleneck` operations, such as authentication and client connection to Google Cloud buckets.  
We will omit them from `Task` to avoid potentially complicated issues debugging that are unrelated to the project.  
In return, execution will be more expensive in terms of consumed RAM and time. Which is a tradeoff that doesn't arise any objections for now.

In [40]:
def process_image_classic(image: Image, image_counter: int, is_tr: bool) -> ProcessedImage:
  width, height = image.size
  if width < MODEL_INPUT["width"] or height < MODEL_INPUT["height"]:
    return None
  if width > MODEL_INPUT["width"] or height > MODEL_INPUT["height"]:
    image.thumbnail((MODEL_INPUT["width"], MODEL_INPUT["height"]))
  if is_tr:
      return ProcessedImage(image_counter, tr_data=[convert_image_to_bytes(image)])
  return ProcessedImage(image_counter, te_data=convert_image_to_bytes(image))

def crop_image_horizontally(image: Image) -> list:
  result = []
  width, height = image.size
  width_ratio = width / MODEL_INPUT["width"]
  if width_ratio <= 1:
    return [image]
  for i in range(int(width_ratio) + 1):
    left_x = i * MODEL_INPUT["width"]
    if left_x >= width:
      break
    right_x = (i + 1) * MODEL_INPUT["width"]
    if right_x > width:
      right_x = width
    result.append(image.crop((left_x, 0, right_x, height)))
  return result

def crop_image_vertically(image: Image) -> list:
  result = []
  width, height = image.size
  height_ratio = height / MODEL_INPUT["height"]
  if height_ratio <= 1:
    return [image]
  for i in range(int(height_ratio) + 1):
    bottom_y = i * MODEL_INPUT["height"]
    if bottom_y >= height:
      break
    upper_y = (i + 1) * MODEL_INPUT["height"]
    if upper_y > height:
      upper_y = height
    result.append(image.crop((0, bottom_y, width, upper_y)))
  return result

def frame_image_in_noise(image: Image) -> Image:
  width, height = image.size
  noise = np.random.normal(loc=MODEL_INPUT["noise_mean"], scale=MODEL_INPUT["noise_std"], size=(MODEL_INPUT["height"], MODEL_INPUT["width"], MODEL_INPUT["color_channels"])).astype(np.uint8)
  framed_image = Image.fromarray(noise)
  top_left_x = (MODEL_INPUT["width"] - width) // 2
  top_left_y = (MODEL_INPUT["height"] - height) // 2
  framed_image.paste(image, (top_left_x, top_left_y))
  return framed_image

def process_image_noise_framed(image: Image, image_counter: int) -> ProcessedImage:
  width, height = image.size
  if width <= MODEL_INPUT["width"] or height <= MODEL_INPUT["height"]:
    framed_image = frame_image_in_noise(image)
    return ProcessedImage(image_counter, tr_data=[convert_image_to_bytes(framed_image)])
  cropped_images = []
  for vertically_cropped_image in crop_image_vertically(image):
    for cropped_image in crop_image_horizontally(vertically_cropped_image):
      cropped_images.append(cropped_image)
  framed_images = [frame_image_in_noise(cropped_image) for cropped_image in cropped_images]
  return ProcessedImage(image_counter, tr_data=[convert_image_to_bytes(framed_image) for framed_image in framed_images])

def process_image(unprocessed_image: UnprocessedImage) -> ProcessedImage:
  image = convert_bytes_to_image(unprocessed_image.data)
  is_tr = unprocessed_image.counter % MODEL_INPUT["te_image_to_tr_images"] == 0
  if FLOW == "classic" or not is_tr:
    return process_image_classic(image, unprocessed_image.counter, is_tr)
  if FLOW == "noise_framed":
    if is_tr:
      return process_image_noise_framed(image, unprocessed_image.counter)
    return process_image_classic(image, unprocessed_image.counter, is_tr)

def preprocessing_task(output_queue: multiprocessing.Queue, batch_of_images: list = []) -> None:
  if output_queue is None or len(batch_of_images) == 0:
    output_queue.put(None)
    return
  results = []
  for unprocessed_image in batch_of_images:
    processed_image = process_image(unprocessed_image)
    if processed_image is None:
      continue
    output_queue.put(processed_image)
  output_queue.put(None)

[Back to Table of Contents](#table-of-contents)

# Main Execution Path
This is where we collect data for the preprocess tasks, run them and store their results.

In [41]:
last_processed_image_idx = 0
while True:
  queue = multiprocessing.Queue()
  processes = []
  processed_batch = []
  processes_outputs_left = 0
  images_batches = get_batches_of_images(client, MACHINE["tasks_number"], last_processed_image_idx, MACHINE["batch_size"])
  if len(images_batches) == 0:
    break
  for batch in images_batches:
    processes.append(multiprocessing.Process(target=preprocessing_task, args=(queue, batch)))
  processes_outputs_left = len(processes)
  for process in processes:
    process.start()
  while processes_outputs_left > 0:
    if queue.empty():
      sleep(2)
      continue
    process_output = queue.get()
    if process_output is None:
      processes_outputs_left -= 1
      continue
    elif type(process_output) is ProcessedImage:
      processed_batch.append(process_output)
    else:
      raise RuntimeError("Unexpected type for output of independent process: %s" % type(process_output))
  for process in processes:
    process.join()
  persist_batch_of_images(client, processed_batch)
  last_processed_image_idx += MACHINE["tasks_number"] * MACHINE["batch_size"]

[Back to Table of Contents](#table-of-contents)